In [65]:
! pip install essentia pandas scikit-learn

  Using cached joblib-1.4.2-py3-none-any.whl.metadata (5.4 kB)
  Using cached threadpoolctl-3.5.0-py3-none-any.whl.metadata (13 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.4/9.4 MB 17.4 MB/s eta 0:00:00a 0:00:01
Using cached joblib-1.4.2-py3-none-any.whl (301 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 28.8/28.8 MB 51.2 MB/s eta 0:00:00a 0:00:01
Using cached threadpoolctl-3.5.0-py3-none-any.whl (18 kB)


In [54]:
import essentia
import essentia.standard as es
import os
from tempfile import TemporaryDirectory

def download_song(artist, song_name, audio_folder='audio', output_folder='song_features'):

    # Ensure audio folder exists
    os.makedirs(audio_folder, exist_ok=True)

    # Format the audio file path
    audio_file = os.path.join(audio_folder, f'{artist}-{song_name}.wav')

    if not os.path.exists(audio_file):
        # If file doesn't exist download the song using spotdl
        os.system(f"spotdl download '{artist}-{song_name}' --format wav --output '{audio_folder}/'")

        # Rename the downloaded file
        audio_file_old = os.path.join(audio_folder, f'{artist} - {song_name}.wav')
        os.rename(audio_file_old, audio_file)

    # Check if file was downloaded
    if not os.path.exists(audio_file):
        raise FileNotFoundError(f"Failed to download or locate audio file: {audio_file}")

    # Ensure output folder exists
    os.makedirs(output_folder, exist_ok=True)

    # Define results file per song
    results_file = os.path.join(output_folder, f'{artist}-{song_name}.json')

    # Extract features
    features, features_frames = es.MusicExtractor(
        lowlevelStats=['mean', 'stdev'],
        rhythmStats=['mean', 'stdev'],
        tonalStats=['mean', 'stdev'])(audio_file)

    # Save features to JSON
    es.YamlOutput(filename=results_file, format='json')(features)

    print(f"Features saved to: {results_file}")
    return results_file


In [59]:
import essentia
import essentia.standard as es
import os
import shutil
from tempfile import TemporaryDirectory

def download_song_2(artist, song_name, audio_folder='audio', output_folder='song_features'):
    # Ensure audio folder exists
    os.makedirs(audio_folder, exist_ok=True)

    # Define the expected audio file path
    expected_audio_file = os.path.join(audio_folder, f'{artist}-{song_name}.wav')

    # Check if the file already exists
    if not os.path.exists(expected_audio_file):
        # Use a temporary directory to download the file
        with TemporaryDirectory() as temp_dir:
            # Download the song using spotdl into the temporary directory
            os.system(f"spotdl download '{artist}-{song_name}' --format wav --output '{temp_dir}/'")

            # Find the downloaded file in the temporary directory
            downloaded_files = os.listdir(temp_dir)
            if not downloaded_files:
                raise FileNotFoundError(f"Failed to download audio file for: {artist}-{song_name}")

            # Assume the first file is the correct one
            downloaded_file = os.path.join(temp_dir, downloaded_files[0])

            # Move and rename the file to the expected path
            shutil.move(downloaded_file, expected_audio_file)

    # Check if the file was successfully downloaded and moved
    if not os.path.exists(expected_audio_file):
        raise FileNotFoundError(f"Failed to locate or rename audio file: {expected_audio_file}")

    # Ensure output folder exists
    os.makedirs(output_folder, exist_ok=True)

    # Define results file per song
    results_file = os.path.join(output_folder, f'{artist}-{song_name}.json')

    # Extract features
    features, features_frames = es.MusicExtractor(
        lowlevelStats=['mean', 'stdev'],
        rhythmStats=['mean', 'stdev'],
        tonalStats=['mean', 'stdev'])(expected_audio_file)

    # Save features to JSON
    es.YamlOutput(filename=results_file, format='json')(features)

    print(f"Features saved to: {results_file}")
    return results_file

In [ ]:
import os
import json
import numpy as np
import pandas as pd
import essentia.standard as es
from tempfile import TemporaryDirectory
from sklearn.preprocessing import StandardScaler

def extract_features_from_json(json_file):
    """
    Extracts relevant features from a JSON file and returns a feature vector.
    """
    with open(json_file, "r") as f:
        data = json.load(f)

    # Extract relevant features
    features = [
        data["lowlevel"]["average_loudness"],
        data["lowlevel"]["dissonance"]["mean"],
        data["lowlevel"]["dynamic_complexity"],
        data["lowlevel"]["spectral_centroid"]["mean"],
        data["lowlevel"]["spectral_flux"]["mean"],
        data["lowlevel"]["zerocrossingrate"]["mean"],
        *data["lowlevel"]["barkbands"]["mean"],  # Flatten barkbands
        *data["lowlevel"]["mfcc"]["mean"],      # Flatten MFCCs
        data["rhythm"]["bpm"],
        data["rhythm"]["beats_count"],
        data["rhythm"]["danceability"],
        data["rhythm"]["onset_rate"],
        data["tonal"]["chords_strength"]["mean"],
        data["tonal"]["hpcp_crest"]["mean"],
        data["tonal"]["hpcp_entropy"]["mean"],
        data["tonal"]["key_edma"]["strength"],
        data["tonal"]["key_krumhansl"]["strength"],
        data["tonal"]["key_temperley"]["strength"],
        data["metadata"]["audio_properties"]["length"],
        data["metadata"]["audio_properties"]["sample_rate"]
    ]

    # Convert to a numpy array
    feature_vector = np.array(features)
    return feature_vector

def save_vector_to_csv(vector, artist, song_name, output_folder="song_csv"):
    """
    Saves the feature vector to a CSV file in the specified output folder.
    """
    # Ensure the output folder exists
    os.makedirs(output_folder, exist_ok=True)

    # Define the CSV file path
    csv_file = os.path.join(output_folder, f"{artist}-{song_name}.csv")

    # Save the vector to a CSV file
    df = pd.DataFrame([vector])
    df.to_csv(csv_file, index=False, header=False)
    print(f"Feature vector saved to: {csv_file}")

def download_and_extract_song(artist, song_name, audio_folder="audio", csv_folder="song_csv", json_folder="song_features"):
    """
    Downloads a song, extracts features, and saves the feature vector to a CSV file.
    """
    # Ensure audio folder exists
    os.makedirs(audio_folder, exist_ok=True)

    # Format the audio file path
    audio_file = os.path.join(audio_folder, f"{artist}-{song_name}.wav")

    if not os.path.exists(audio_file):
        # If file doesn't exist, download the song using spotdl
        os.system(f"spotdl download '{artist}-{song_name}' --format wav --output '{audio_folder}/'")

        # Rename the downloaded file
        downloaded_files = [f for f in os.listdir(audio_folder) if f.endswith(".wav")]
        if downloaded_files:
            downloaded_file = os.path.join(audio_folder, downloaded_files[0])
            os.rename(downloaded_file, audio_file)

    # Check if file was downloaded
    if not os.path.exists(audio_file):
        raise FileNotFoundError(f"Failed to download or locate audio file: {audio_file}")

    # Ensure output folder exists
    os.makedirs(json_folder, exist_ok=True)

    # Define results file per song
    results_file = os.path.join(json_folder, f"{artist}-{song_name}.json")

    # Extract features using Essentia
    features, features_frames = es.MusicExtractor(
        lowlevelStats=['mean', 'stdev'],
        rhythmStats=['mean', 'stdev'],
        tonalStats=['mean', 'stdev'])(audio_file)

    # Save features to JSON
    es.YamlOutput(filename=results_file, format='json')(features)

    # Extract the feature vector from the JSON file
    feature_vector = extract_features_from_json(results_file)

    # Save the feature vector to a CSV file
    save_vector_to_csv(feature_vector, artist, song_name, csv_folder)

    return feature_vector


Feature vector saved to: song_csv/Vaundy-Odoriko.csv


[   INFO   ] MusicExtractor: Read metadata
[   INFO   ] MusicExtractor: Compute md5 audio hash, codec, length, and EBU 128 loudness
[   INFO   ] MusicExtractor: Replay gain
[   INFO   ] MusicExtractor: Compute audio features
[   INFO   ] MusicExtractor: Compute aggregation
[   INFO   ] All done


array([ 9.39152718e-01,  4.16007102e-01,  4.12004185e+00,  4.23429565e+02,
        9.02605876e-02,  1.90045405e-02,  1.93124855e-04,  1.13086207e-02,
        9.16276313e-03,  5.08838845e-03,  9.04331822e-03,  2.81741167e-03,
        2.57824338e-03,  1.31346891e-03,  1.13959203e-03,  6.51220209e-04,
        2.15309847e-04,  2.11447492e-04,  1.85756886e-04,  1.86710546e-04,
        1.15832991e-04,  8.52811499e-05,  6.75560586e-05,  3.26277477e-05,
        2.29881152e-05,  2.30401420e-05,  1.36801909e-05,  8.54081009e-06,
        5.10861264e-06,  3.48543540e-06,  2.24170753e-06,  9.58161991e-07,
        1.77022429e-07, -7.67552002e+02,  2.06609314e+02,  1.69044514e+01,
        3.04737225e+01,  1.24275017e+01, -2.65572262e+00, -3.14775491e+00,
       -3.83518195e+00,  1.17408383e+00,  3.22530150e-01, -8.85620213e+00,
       -3.98777276e-01, -4.19929600e+00,  1.56233597e+02,  6.00000000e+02,
        1.03065944e+00,  4.65406609e+00,  5.21835387e-01,  1.38977060e+01,
        1.88031411e+00,  

In [68]:
download_and_extract_song('Gen Hoshino', 'Comedy')

Feature vector saved to: song_csv/Gen Hoshino-Comedy.csv


[   INFO   ] MusicExtractor: Read metadata
[   INFO   ] MusicExtractor: Compute md5 audio hash, codec, length, and EBU 128 loudness
[   INFO   ] MusicExtractor: Replay gain
[   INFO   ] MusicExtractor: Compute audio features
[   INFO   ] MusicExtractor: Compute aggregation
[   INFO   ] All done


array([ 8.73837113e-01,  4.44118559e-01,  3.73841119e+00,  1.12656702e+03,
        1.04242519e-01,  5.15120253e-02,  1.59507466e-03,  3.07862218e-02,
        6.83374889e-03,  2.66077602e-03,  3.37207830e-03,  2.01089494e-03,
        1.57965312e-03,  5.90952055e-04,  9.56705830e-04,  8.47762625e-04,
        3.73872172e-04,  1.67846927e-04,  3.58659483e-04,  4.66708210e-04,
        1.86761725e-04,  1.74043060e-04,  1.68964529e-04,  9.15279816e-05,
        2.28002944e-04,  2.11922219e-04,  1.68714483e-04,  1.68223036e-04,
        1.17100455e-04,  8.42383452e-05,  8.78756327e-05,  3.90601454e-05,
        1.22988049e-05, -6.85507568e+02,  1.20436150e+02,  3.01479492e+01,
        2.58509312e+01, -4.05921555e+00,  5.17231083e+00, -3.90065819e-01,
       -2.72737193e+00,  2.59658408e+00,  2.99214983e+00,  2.16747206e-02,
       -4.22734594e+00, -6.06712401e-01,  8.80438232e+01,  3.36000000e+02,
        1.37919295e+00,  3.88852668e+00,  4.84918505e-01,  1.39006453e+01,
        1.90955162e+00,  